<a href="https://colab.research.google.com/github/Kei-Sanada/Numerai/blob/master/234_Modeling_20201019.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# install dependencies
!pip install pandas sklearn numerapi

In [ ]:
import pandas as pd
import numerapi

In [ ]:


# training data contains features and targets
#training_data = pd.read_csv("numerai_training_data.csv").set_index("id")
# download the latest training dataset (takes around 30s)
#training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz").set_index("id")
training_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_training_data.csv.xz")
training_data.head()

,id,era,data_type,feature_intelligence1,feature_intelligence2,feature_intelligence3,feature_intelligence4,feature_intelligence5,feature_intelligence6,feature_intelligence7,feature_intelligence8,feature_intelligence9,feature_intelligence10,feature_intelligence11,feature_intelligence12,feature_charisma1,feature_charisma2,feature_charisma3,feature_charisma4,feature_charisma5,feature_charisma6,feature_charisma7,feature_charisma8,feature_charisma9,feature_charisma10,feature_charisma11,feature_charisma12,feature_charisma13,feature_charisma14,feature_charisma15,feature_charisma16,feature_charisma17,feature_charisma18,feature_charisma19,feature_charisma20,feature_charisma21,feature_charisma22,feature_charisma23,feature_charisma24,feature_charisma25,...,feature_wisdom8,feature_wisdom9,feature_wisdom10,feature_wisdom11,feature_wisdom12,feature_wisdom13,feature_wisdom14,feature_wisdom15,feature_wisdom16,feature_wisdom17,feature_wisdom18,feature_wisdom19,feature_wisdom20,feature_wisdom21,feature_wisdom22,feature_wisdom23,feature_wisdom24,feature_wisdom25,feature_wisdom26,feature_wisdom27,feature_wisdom28,feature_wisdom29,feature_wisdom30,feature_wisdom31,feature_wisdom32,feature_wisdom33,feature_wisdom34,feature_wisdom35,feature_wisdom36,feature_wisdom37,feature_wisdom38,feature_wisdom39,feature_wisdom40,feature_wisdom41,feature_wisdom42,feature_wisdom43,feature_wisdom44,feature_wisdom45,feature_wisdom46,target_kazutsugi
0,n000315175b67977,era1,train,0.00,0.50,0.25,0.00,0.5,0.25,0.25,0.25,0.75,0.75,0.25,0.25,1.00,0.75,0.5,1.0,0.5,0.00,0.50,0.50,0.00,0.0,0.00,1.00,0.25,0.00,0.50,0.25,0.75,0.50,1.00,0.75,0.75,0.5,0.50,0.75,0.5,...,0.75,0.75,0.75,0.50,1.00,1.00,0.50,0.75,0.5,0.25,0.25,0.75,0.50,1.00,0.5,0.75,0.75,0.25,0.50,1.00,0.75,0.50,0.50,1.00,0.25,0.5,0.50,0.50,0.75,1.00,1.00,1.00,0.75,0.50,0.75,0.50,1.00,0.50,0.75,0.75
1,n0014af834a96cdd,era1,train,0.00,0.00,0.00,0.25,0.5,0.00,0.00,0.25,0.50,0.50,0.00,0.50,0.00,0.50,0.5,0.5,0.5,0.50,0.25,0.25,0.50,0.0,1.00,0.50,0.50,0.50,0.75,0.50,0.50,0.75,0.25,0.50,0.75,0.5,0.25,0.75,0.5,...,0.25,0.25,0.25,1.00,1.00,0.50,0.50,0.50,0.0,0.25,1.00,0.50,1.00,1.00,0.5,0.50,0.50,1.00,0.25,0.75,1.00,0.25,0.25,1.00,0.50,0.5,0.50,0.75,0.75,0.75,1.00,1.00,0.00,0.00,0.75,0.25,0.00,0.25,1.00,0.25
2,n001c93979ac41d4,era1,train,0.25,0.50,0.25,0.25,1.0,0.75,0.75,0.25,0.00,0.25,0.50,1.00,0.50,0.75,0.5,0.5,1.0,0.50,0.50,0.50,0.25,0.0,0.25,0.75,0.75,0.75,0.50,0.75,0.50,0.25,0.50,0.75,0.25,0.5,0.50,0.75,0.5,...,0.25,1.00,1.00,1.00,0.50,1.00,1.00,1.00,0.5,1.00,0.00,1.00,1.00,0.50,1.0,0.75,1.00,0.00,0.50,0.75,0.00,1.00,0.50,0.50,0.75,1.0,0.75,1.00,0.25,0.50,0.25,0.50,0.00,0.00,0.50,1.00,0.00,0.25,0.75,0.00
3,n0034e4143f22a13,era1,train,1.00,0.00,0.00,0.50,0.5,0.25,0.25,0.75,0.25,0.50,0.50,0.50,0.75,0.50,1.0,0.5,0.5,0.00,1.00,0.00,0.75,0.0,0.50,0.50,0.50,0.50,0.00,0.50,0.50,0.75,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,1.00,0.75,0.75,1.00,0.75,0.75,0.75,1.0,0.75,1.00,0.75,1.00,0.75,1.0,0.00,0.50,0.75,1.00,0.75,1.00,0.75,1.00,1.00,0.00,0.5,0.75,0.75,1.00,0.75,1.00,1.00,0.75,0.75,1.00,1.00,0.75,1.00,1.00,0.00
4,n00679d1a636062f,era1,train,0.25,0.25,0.25,0.25,0.0,0.25,0.50,0.25,0.25,0.50,0.25,0.25,0.75,0.50,0.0,0.5,0.5,0.25,0.00,0.50,0.00,0.5,0.25,0.50,0.50,0.50,0.50,0.50,0.50,0.50,0.75,0.50,0.25,0.5,0.50,0.50,0.5,...,1.00,0.25,0.75,1.00,0.75,0.00,0.00,0.75,0.5,1.00,0.50,0.75,0.25,0.50,0.0,0.50,0.50,0.50,0.75,0.75,0.50,0.75,0.25,0.75,0.50,0.5,0.25,0.25,0.75,0.50,0.75,0.75,0.25,0.50,0.75,0.00,0.50,0.25,0.75,0.75


In [ ]:
# training data contains features and targets
#tournament_data = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz").set_index("id")
tournament_data  = pd.read_csv("https://numerai-public-datasets.s3-us-west-2.amazonaws.com/latest_numerai_tournament_data.csv.xz")


230 and 233

In [ ]:
# import dependencies
import sklearn.linear_model

In [ ]:
# find only the feature columns
feature_cols = training_data.columns[training_data.columns.str.startswith('feature_wisdom')]

# select those columns out of the training dataset
training_features = training_data[feature_cols]

In [ ]:
# create a model and fit the training data (~30 sec to run)
model = sklearn.linear_model.LinearRegression()
model.fit(training_features, training_data.target_kazutsugi)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [ ]:
# select the feature columns from the tournament data
live_features = tournament_data[feature_cols]

In [ ]:
# predict the target on the live features
sklearn_linear_model_predictions = model.predict(live_features)

In [ ]:
feature_names = [f for f in training_data.columns if "feature_dexterity" in f]
feature_names

['feature_dexterity1',
 'feature_dexterity2',
 'feature_dexterity3',
 'feature_dexterity4',
 'feature_dexterity5',
 'feature_dexterity6',
 'feature_dexterity7',
 'feature_dexterity8',
 'feature_dexterity9',
 'feature_dexterity10',
 'feature_dexterity11',
 'feature_dexterity12',
 'feature_dexterity13',
 'feature_dexterity14']

In [ ]:
 training_data["target_kazutsugi"]

0         0.75
1         0.25
2         0.00
3         0.00
4         0.75
          ... 
501803    0.25
501804    1.00
501805    0.00
501806    0.25
501807    0.50
Name: target_kazutsugi, Length: 501808, dtype: float64

In [ ]:
from xgboost import XGBRegressor

# train a model to make predictions on tournament data
#model = XGBRegressor(max_depth=5, learning_rate=0.01, n_estimators=2000, colsample_bytree=0.1)
model = XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bynode=1, colsample_bytree=1, gamma=0,
       importance_type='gain', learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='reg:squarederror', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
       silent=None, subsample=1, verbosity=1)
model.fit(training_data[feature_names], training_data["target_kazutsugi"])

# submit predictions to numer.ai
predictions_XGBRegressor = model.predict(tournament_data[feature_names])

In [ ]:
predictions = (sklearn_linear_model_predictions + predictions_XGBRegressor) / 2

In [ ]:
tournament_data["id"]

0          n0003aa52cab36c2
1          n000920ed083903f
2          n0038e640522c4a6
3          n004ac94a87dc54b
4          n0052fe97ea0c05f
                 ...       
1663694    nff9ec15b1a7f678
1663695    nffa3e54e6ebf447
1663696    nffb929067fbd1e7
1663697    nfff41c66c661138
1663698    nfff520453843ace
Name: id, Length: 1663699, dtype: object

In [ ]:
# predictions must have an `id` column and a `prediction_kazutsugi` column
predictions_df = tournament_data["id"].to_frame()
predictions_df["prediction_kazutsugi"] = predictions
predictions_df.head()

,id,prediction_kazutsugi
0,n0003aa52cab36c2,0.493514
1,n000920ed083903f,0.505815
2,n0038e640522c4a6,0.503612
3,n004ac94a87dc54b,0.493159
4,n0052fe97ea0c05f,0.494989


In [ ]:
import numpy as np
ranked_prediction = training_data["target_kazutsugi"].rank(pct=True, method="first")
correlation = np.corrcoef(training_data["target_kazutsugi"], ranked_prediction)[0, 1]

In [ ]:
ranked_prediction

0         0.601268
1         0.199365
2         0.000002
3         0.000004
4         0.601270
            ...   
501803    0.398732
501804    1.000000
501805    0.199363
501806    0.398734
501807    0.601266
Name: target_kazutsugi, Length: 501808, dtype: float64

In [ ]:
correlation

0.9797922144295415

In [ ]:
training_data["target_kazutsugi"]

0         0.75
1         0.25
2         0.00
3         0.00
4         0.75
          ... 
501803    0.25
501804    1.00
501805    0.00
501806    0.25
501807    0.50
Name: target_kazutsugi, Length: 501808, dtype: float64

In [ ]:
# Get your API keys and model_id from https://numer.ai/submit
public_id = "CYATEL5QQBU6APNFLCV7HEE7PV6SC7V6"
secret_key = "Y22BTSUGU4JEFGQB3RZNEESSULKA3HQJPAW3KI6BIXH2AMNMCTC44IFWTOQIO2UW"
model_id = "3c77ba09-cfa2-4b18-b789-918340c84c82"
napi = numerapi.NumerAPI(public_id=public_id, secret_key=secret_key)

In [ ]:
predictions_df.to_csv("predictions.csv", index=False)
submission_id = napi.upload_predictions("predictions.csv", model_id=model_id)

2020-10-19 14:56:33,843 INFO numerapi.base_api: uploading predictions...
